In [1]:
import pandas as pd
import torch

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\User\Desktop\programowanie_web_etc\python_projects\data_projects\venv\lib\site-packages\transformers\models\auto\modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [3]:
def get_emotion(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)
  
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label

In [4]:
get_emotion("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'

'<pad> joy'

In [5]:
get_emotion("i have a feeling i kinda lost my best friend") # Output: 'sadness'

'<pad> sadness'

In [6]:
df_path = 'dataframes/with_counts/combined_count.csv'

In [7]:
df = pd.read_csv(df_path, index_col = 0)

In [8]:
sample_lyrics = df.sample(5)['Song Lyrics'].str.lower()

In [9]:
import re
emotion_regex = re.compile('<pad>\s(.*)')
emotion_regex.findall('<pad> sadness')[0]

'sadness'

In [10]:
for song in sample_lyrics:
    song_emotion = get_emotion(song)
    print('song:')
    print(song)
    print('===================================')
    print('emotion:')
    print(song_emotion)
    print('===================================')
    print('===================================')

Token indices sequence length is longer than the specified maximum sequence length for this model (937 > 512). Running this sequence through the model will result in indexing errors


song:
 if you ever change your mind about leaving leaving me behind  baby bring it to me bring your sweet loving bring it on home to me yeah yeah yeah yeah yeah yeah  i know i laughed when you left but now i know i only hurt myself  baby bring it to me bring your sweet loving bring it on home to me yeah yeah yeah yeah yeah yeah  ill give you jewelry and money too that aint all that aint all ill do for you  baby if you bring it to me bring your sweet loving bring it on home to me yeah yeah yeah yeah yeah yeah you might also like you know ill always be your slave til im buried buried in my grave  oh honey bring it to me bring your sweet loving bring it on home to me yeah yeah yeah yeah yeah yeah  one more thing i tried to treat you right but you stayed out stayed out day and night  but i forgive you bring it to me bring your sweet loving bring it on home to me yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah yeah
emotion:
<pad> joy
song:

In [11]:
def assign_emotion(row):
    lyrics = row['Song Lyrics']
    emotion = get_emotion(lyrics)
    emotion = emotion_regex.findall(emotion)[0]
    return emotion

In [12]:
def apply_emotion(df):
    df['emotion'] = df.apply(
        lambda row: assign_emotion(row),
        axis = 1
    )
    return df

In [13]:
df = apply_emotion(df)

In [14]:
df.sample(5)

,Artist,Song Title,Full Title,Release Date,Year,Month,Day,Pageviews,url,featured_count,...,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,genre,emotion
78,Carly Rae Jepsen,Everywhere You Look (The Fuller House Theme),Everywhere You Look (The Fuller House Theme) b...,2016-02-26,2016.0,2.0,26.0,36765,https://genius.com/Carly-rae-jepsen-everywhere...,0,...,5,4,26,14,1,1,1,1,pop,fear
446,Foo Fighters,The Sky Is a Neighborhood,The Sky Is a Neighborhood by Foo Fighters,2017-08-23,2017.0,8.0,23.0,201863,https://genius.com/Foo-fighters-the-sky-is-a-n...,0,...,5,4,26,14,1,1,1,1,rap,sadness
276,Ice Cube,Friday,Friday by Ice Cube,1995-04-11,1995.0,4.0,11.0,61913,https://genius.com/Ice-cube-friday-lyrics,0,...,5,4,26,14,1,1,1,1,rock,joy
294,JAY-Z,Family Feud,Family Feud by JAY-Z (Ft. Beyoncé),2017-06-30,2017.0,6.0,30.0,944810,https://genius.com/Jay-z-family-feud-lyrics,1,...,5,4,26,14,1,1,1,1,rock,joy
254,Eminem,Superman,Superman by Eminem (Ft. Dina Rae),2002-05-26,2002.0,5.0,26.0,3015114,https://genius.com/Eminem-superman-lyrics,1,...,5,4,26,14,1,1,1,1,rock,anger


In [15]:
df.to_csv('dataframes/with_counts/combined_count.csv')